<a href="https://colab.research.google.com/github/fix27/Colab-Store/blob/main/access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt update
!ACCEPT_EULA=Y apt -q -y install msodbcsql17
!pip install pyodbc

import pyodbc
import pandas as pd

# Параметры подключения к базе данных Access
db_path = r'/content/slalom2020.mdb'  # Укажите путь к вашему файлу .accdb
conn_str = r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + db_path + ';'

# Подключение к базе данных
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Получение списка всех таблиц в базе данных
tables = cursor.tables(tableType='TABLE')
table_names = [table.table_name for table in tables]

# Получение списка всех представлений (запросов)
views_query = """
SELECT Name, SQL
FROM MSysObjects
WHERE Type = 5 AND Flags = 0  -- Type = 5 означает запросы (представления)
ORDER BY Name;
"""
cursor.execute(views_query)
views = cursor.fetchall()

# Получение списка всех системных таблиц
system_tables_query = """
SELECT Name, Type
FROM MSysObjects
WHERE Type = 1 AND Flags = 0  -- Type = 1 означает системные таблицы
ORDER BY Name;
"""
cursor.execute(system_tables_query)
system_tables = cursor.fetchall()

# Открытие файла для записи SQL-запросов
with open('full_export.sql', 'w', encoding='utf-8') as sql_file:
    # Выгрузка данных из таблиц
    for table_name in table_names:
        # Получение данных из таблицы
        query = f"SELECT * FROM [{table_name}]"
        df = pd.read_sql(query, conn)

        # Генерация SQL-запросов для вставки данных
        sql_file.write(f"-- Table: {table_name}\n")
        for _, row in df.iterrows():
            columns = ', '.join([f'[{col}]' for col in df.columns])
            values = ', '.join([f"""'{str(value).replace("'", "''")}'""" if isinstance(value, str) else str(value) for value in row])
            insert_query = f"INSERT INTO [{table_name}] ({columns}) VALUES ({values});\n"
            sql_file.write(insert_query)
        sql_file.write("\n-- ----------------------------\n\n")

    # Выгрузка представлений
    for view in views:
        view_name, view_sql = view
        sql_file.write(f"-- View: {view_name}\n")
        sql_file.write(f"CREATE VIEW [{view_name}] AS\n")
        sql_file.write(view_sql + ";\n\n")
        sql_file.write("-- ----------------------------\n\n")

    # Выгрузка системных таблиц
    for system_table in system_tables:
        table_name, table_type = system_table
        sql_file.write(f"-- System Table: {table_name}\n")
        sql_file.write(f"-- Type: {table_type}\n")
        sql_file.write("-- ----------------------------\n\n")

# Закрытие соединения
cursor.close()
conn.close()

print("Данные успешно выгружены в файл full_export.sql")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100   983  100   983    0     0   2926      0 --:--:-- --:--:-- --:--:--  2934
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    78  100    78    0     0    387      0 --:--:-- --:--:-- --:--:--   388
Get:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease [3,638 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 

Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'Microsoft Access Driver (*.mdb, *.accdb)' : file not found (0) (SQLDriverConnect)")